In [17]:
import numpy as np
import scipy.linalg
from scipy.optimize import minimize
from scipy.io import loadmat # For SARCOS
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- Kernel Function ---
class SquaredExponentialKernel:
    def __init__(self, length_scale=1.0, sigma_f=1.0):
        self.length_scale = length_scale
        self.sigma_f = sigma_f

    def get_params(self):
        return np.array([self.length_scale, self.sigma_f])

    def set_params(self, params):
        self.length_scale = params[0]
        self.sigma_f = params[1]

    def __call__(self, X1, X2):
        if X1.ndim == 1: X1 = X1[:, np.newaxis]
        if X2.ndim == 1: X2 = X2[:, np.newaxis]

        if X1.shape[1] != X2.shape[1]:
            raise ValueError(f"X1 and X2 must have the same number of features. Got {X1.shape[1]} and {X2.shape[1]}")

        sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
        sqdist = np.clip(sqdist, 0, np.inf) # Ensure non-negative for sqrt if used elsewhere

        return self.sigma_f**2 * np.exp(-0.5 / self.length_scale**2 * sqdist)

# Helper to compute covariance matrix
def calculate_covariance_matrix(X1, X2, kernel_func):
    return kernel_func(X1, X2)

# --- NMSE Metric ---
def nmse(y_true, y_pred):
    y_true_v = np.asarray(y_true).flatten()
    y_pred_v = np.asarray(y_pred).flatten()
    if len(y_true_v) == 0 or len(y_pred_v) == 0: return np.nan
    mse = np.mean((y_true_v - y_pred_v)**2)
    var_true = np.var(y_true_v)
    if var_true < 1e-9: # Avoid division by zero if true values are constant
        return mse if mse > 1e-9 else 0.0
    return mse / var_true

# --- AT-GP Model (from paper) ---
class ATGP:
    def __init__(self, X_S, y_S, X_T, y_T,
                 initial_length_scale=1.0, initial_sigma_f=1.0,
                 initial_b=1.0, initial_mu=1.0,
                 initial_noise_S_std=0.1, initial_noise_T_std=0.1):
        self.X_S = X_S
        self.y_S = y_S.reshape(-1, 1)
        self.X_T = X_T
        self.y_T = y_T.reshape(-1, 1)

        self._length_scale = initial_length_scale
        self._sigma_f = initial_sigma_f
        self._b = initial_b
        self._mu = initial_mu
        self._noise_S_std = initial_noise_S_std
        self._noise_T_std = initial_noise_T_std

        self.base_kernel = SquaredExponentialKernel(self._length_scale, self._sigma_f)
        self.fitted = False
        self.optimized_params_ = {}

    def _get_params_array(self):
        return np.array([self._length_scale, self._sigma_f, self._b, self._mu,
                         self._noise_S_std, self._noise_T_std])

    def _set_params_from_array(self, params_array):
        self._length_scale = params_array[0]
        self._sigma_f = params_array[1]
        self.base_kernel.set_params(params_array[:2])
        self._b = params_array[2]
        self._mu = params_array[3]
        self._noise_S_std = params_array[4]
        self._noise_T_std = params_array[5]

    def _get_lambda(self, b_param=None, mu_param=None):
        b_to_use = b_param if b_param is not None else self._b
        mu_to_use = mu_param if mu_param is not None else self._mu

        # Parameter constraints from practical application (and to avoid math errors)
        b_to_use = max(1e-5, b_to_use)
        mu_to_use = max(1e-5, mu_to_use) # mu > -1, but for (1+mu) often > 0 is better

        term_base = 1.0 / (1.0 + mu_to_use)
        lambda_val = -1.0
        if term_base < 0 : # (1+mu) is negative
             lambda_val = -1.0 # Or some other defined behavior
        elif abs(term_base) > 1e6 and b_to_use > 1 and term_base > 0: # Avoid large number to large power
            lambda_val = 1.0
        else:
            try:
                pow_term = np.power(term_base, b_to_use)
                lambda_val = 2 * pow_term - 1
            except (OverflowError, ValueError):
                lambda_val = 1.0 if term_base > 1 else -1.0 # Fallback for extreme values

        return np.clip(lambda_val, -1.0, 1.0)


    def log_marginal_likelihood_conditional(self, params_array_opt):
        # Ensure parameters are positive where necessary
        params_array_opt = np.maximum(params_array_opt, 1e-5)

        current_length_scale = params_array_opt[0]
        current_sigma_f = params_array_opt[1]
        temp_kernel = SquaredExponentialKernel(current_length_scale, current_sigma_f)

        current_b = params_array_opt[2]
        current_mu = params_array_opt[3]
        temp_lambda = self._get_lambda(b_param=current_b, mu_param=current_mu)

        current_noise_S_var = params_array_opt[4]**2
        current_noise_T_var = params_array_opt[5]**2

        jitter = 1e-7 # For numerical stability

        K11 = calculate_covariance_matrix(self.X_S, self.X_S, temp_kernel)
        K22 = calculate_covariance_matrix(self.X_T, self.X_T, temp_kernel)
        K_TS_base = calculate_covariance_matrix(self.X_T, self.X_S, temp_kernel)

        K21 = temp_lambda * K_TS_base
        K12 = K21.T # Or temp_lambda * K_TS_base.T

        try:
            K11_noisy = K11 + current_noise_S_var * np.eye(K11.shape[0]) + jitter * np.eye(K11.shape[0])
            L_K11_noisy = np.linalg.cholesky(K11_noisy)

            # inv(K11_noisy) @ y_S
            K11_noisy_inv_yS = scipy.linalg.solve_triangular(L_K11_noisy.T, scipy.linalg.solve_triangular(L_K11_noisy, self.y_S, lower=True, check_finite=False), lower=False, check_finite=False)
            mu_t = K21 @ K11_noisy_inv_yS

            # inv(K11_noisy) @ K12
            K11_noisy_inv_K12 = scipy.linalg.solve_triangular(L_K11_noisy.T, scipy.linalg.solve_triangular(L_K11_noisy, K12, lower=True, check_finite=False), lower=False, check_finite=False)

            C_t_main_term = K22 - K21 @ K11_noisy_inv_K12
            C_t = C_t_main_term + current_noise_T_var * np.eye(K22.shape[0]) + jitter * np.eye(K22.shape[0])

            L_Ct = np.linalg.cholesky(C_t)
            log_det_Ct = 2 * np.sum(np.log(np.diag(L_Ct)))

            y_T_minus_mu_t = self.y_T - mu_t
            # inv(C_t) @ (y_T - mu_t)
            Ct_inv_y_minus_mu = scipy.linalg.solve_triangular(L_Ct.T, scipy.linalg.solve_triangular(L_Ct, y_T_minus_mu_t, lower=True, check_finite=False), lower=False, check_finite=False)

            term2_quadratic = y_T_minus_mu_t.T @ Ct_inv_y_minus_mu
        except np.linalg.LinAlgError:
            return -np.inf # Penalize if Cholesky fails

        log_likelihood = -0.5 * log_det_Ct - 0.5 * term2_quadratic - len(self.y_T)/2.0 * np.log(2 * np.pi)

        if np.isnan(log_likelihood) or not np.isfinite(log_likelihood):
            return -np.inf
        return log_likelihood.item()

    def fit(self, method='L-BFGS-B', disp=False, maxiter=200):
        initial_params = self._get_params_array()
        if disp:
            print("  Starting AT-GP optimization...")
            print(f"    Initial AT-GP params (ls, sf, b, mu, noise_S, noise_T): {np.round(initial_params,3)}")
            print(f"    Initial AT-GP lambda: {self._get_lambda():.3f}")

        bounds = [
            (1e-5, 1e3),  # length_scale
            (1e-5, 1e3),  # sigma_f
            (1e-5, 1e2),  # b
            (1e-5, 1e2),  # mu (mu > -1 for (1+mu) to be positive in (1/(1+mu)))
            (1e-5, 1e2),  # noise_S_std
            (1e-5, 1e2)   # noise_T_std
        ]

        objective = lambda params_opt: -self.log_marginal_likelihood_conditional(params_opt)

        result = minimize(objective, initial_params, method=method, bounds=bounds, options={'disp': disp, 'maxiter': maxiter, 'ftol': 1e-7, 'gtol': 1e-5})

        if disp:
            print("  AT-GP optimization finished.")
            print(f"    Success: {result.success}, Message: {result.message}")
            print(f"    Optimized AT-GP params (ls, sf, b, mu, noise_S, noise_T): {np.round(result.x,3)}")


        if result.success or "CONVERGENCE" in result.message.upper() :
            self._set_params_from_array(result.x)
            self.fitted = True
            self.optimized_params_ = dict(zip(['ls', 'sf', 'b', 'mu', 'noise_S', 'noise_T', 'lambda'],
                                           list(np.round(result.x,4)) + [np.round(self._get_lambda(),4)]))
            if disp:
                 print(f"    Optimized AT-GP lambda: {self._get_lambda():.3f}")
        else:
            if disp:
                print(f"    Warning: AT-GP Optimization may not have fully converged: {result.message}")
            # Still set params to the best found
            self._set_params_from_array(result.x)
            self.fitted = True
            self.optimized_params_ = dict(zip(['ls', 'sf', 'b', 'mu', 'noise_S', 'noise_T', 'lambda'],
                                           list(np.round(result.x,4)) + [np.round(self._get_lambda(),4)]))
            if disp:
                 print(f"    Resulting AT-GP lambda (from potentially non-converged params): {self._get_lambda():.3f}")
        return result

    def predict(self, X_star_T):
        if not self.fitted :
             print("Warning: ATGP model is not fitted. Predictions based on initial parameters.")

        lambda_val = self._get_lambda()
        noise_S_var_val = self._noise_S_std**2
        noise_T_var_val = self._noise_T_std**2
        jitter = 1e-7

        N_S = self.X_S.shape[0]
        N_T = self.X_T.shape[0]
        N_all = N_S + N_T

        # Construct the full data matrices for the combined system (Eq 7 context)
        # X_all = np.vstack((self.X_S, self.X_T)) # Not explicitly needed for K_tilde parts
        y_all = np.vstack((self.y_S, self.y_T))

        K_SS = calculate_covariance_matrix(self.X_S, self.X_S, self.base_kernel)
        K_TT = calculate_covariance_matrix(self.X_T, self.X_T, self.base_kernel)
        K_ST_base = calculate_covariance_matrix(self.X_S, self.X_T, self.base_kernel) # K_S_TargTrain

        K_tilde = np.zeros((N_all, N_all))
        K_tilde[:N_S, :N_S] = K_SS
        K_tilde[N_S:, N_S:] = K_TT
        K_tilde[:N_S, N_S:] = lambda_val * K_ST_base
        K_tilde[N_S:, :N_S] = lambda_val * K_ST_base.T

        Big_Lambda_diag = np.concatenate([np.full(N_S, noise_S_var_val),
                                          np.full(N_T, noise_T_var_val)])
        Big_Lambda = np.diag(Big_Lambda_diag)

        C_tilde = K_tilde + Big_Lambda + jitter * np.eye(N_all) # This is C in Eq 7
        L_Ctilde = None
        try:
            L_Ctilde = np.linalg.cholesky(C_tilde)
            # alpha = C_tilde_inv @ y_all
            alpha = scipy.linalg.solve_triangular(L_Ctilde.T, scipy.linalg.solve_triangular(L_Ctilde, y_all, lower=True, check_finite=False), lower=False, check_finite=False)
        except np.linalg.LinAlgError:
            # Fallback to pseudo-inverse if Cholesky fails
            C_tilde_inv = np.linalg.pinv(C_tilde)
            alpha = C_tilde_inv @ y_all

        # k_x in Eq 7 (covariance between test point X_star_T and all training data)
        k_star_S_base = calculate_covariance_matrix(X_star_T, self.X_S, self.base_kernel) # K_Test_Source
        k_star_T_base = calculate_covariance_matrix(X_star_T, self.X_T, self.base_kernel) # K_Test_TargTrain
        # The paper implies kx has lambda for source part, 1 for target part when predicting for target
        k_x_star_rows = np.hstack((lambda_val * k_star_S_base, k_star_T_base))

        mean_star = k_x_star_rows @ alpha # m(x) = kx C̃⁻¹ y (Eq 7)

        # c in Eq 7 is k(x,x) + beta_t^-1 (target noise variance)
        # k(x,x) for test points
        k_star_star_diag = np.diag(self.base_kernel(X_star_T, X_star_T))
        c_diag = k_star_star_diag + noise_T_var_val # This is 'c' in sigma^2(x) = c - kx C̃⁻¹ kxᵀ

        if L_Ctilde is not None:
            # v = L_Ctilde^-1 @ k_x_star_rows.T
            v = scipy.linalg.solve_triangular(L_Ctilde, k_x_star_rows.T, lower=True, check_finite=False)
            var_reduction_diag = np.sum(v**2, axis=0) # diag(kx C̃⁻¹ kxᵀ)
        else:
             # If Cholesky failed, use the pseudo-inverse C_tilde_inv
             var_reduction_diag = np.diag(k_x_star_rows @ C_tilde_inv @ k_x_star_rows.T)


        variance_star_diag = c_diag - var_reduction_diag
        variance_star_diag = np.clip(variance_star_diag, jitter, np.inf) # Ensure positive variance

        return mean_star, variance_star_diag.reshape(-1,1)


# --- Simple GPR for Baselines ---
class SimpleGPR:
    def __init__(self, initial_length_scale=1.0, initial_sigma_f=1.0, initial_noise_std=0.1):
        self._length_scale = initial_length_scale
        self._sigma_f = initial_sigma_f
        self._noise_std = initial_noise_std
        self.kernel = SquaredExponentialKernel(self._length_scale, self._sigma_f)
        self.X_train, self.y_train, self.alpha_, self.L_ = None, None, None, None
        self.fitted = False
        self.optimized_params_ = {}


    def _get_params_array(self):
        return np.array([self._length_scale, self._sigma_f, self._noise_std])

    def _set_params_from_array(self, params_array):
        self._length_scale = params_array[0]
        self._sigma_f = params_array[1]
        self.kernel.set_params(params_array[:2])
        self._noise_std = params_array[2]

    def log_marginal_likelihood(self, params_array_opt, X, y):
        params_array_opt = np.maximum(params_array_opt, 1e-5) # Ensure positive
        current_length_scale, current_sigma_f = params_array_opt[0], params_array_opt[1]
        temp_kernel = SquaredExponentialKernel(current_length_scale, current_sigma_f)
        current_noise_var = params_array_opt[2]**2
        jitter = 1e-7

        N = X.shape[0]
        K = calculate_covariance_matrix(X, X, temp_kernel)
        K_noisy = K + current_noise_var * np.eye(N) + jitter * np.eye(N)

        try:
            L = np.linalg.cholesky(K_noisy)
            # alpha_solve = inv(K_noisy) @ y
            alpha_solve = scipy.linalg.solve_triangular(L.T, scipy.linalg.solve_triangular(L, y, lower=True, check_finite=False), lower=False, check_finite=False)
            log_det_K_noisy = 2 * np.sum(np.log(np.diag(L)))
        except np.linalg.LinAlgError: return -np.inf

        lml = -0.5 * y.T @ alpha_solve - 0.5 * log_det_K_noisy - N/2.0 * np.log(2 * np.pi)
        if np.isnan(lml) or not np.isfinite(lml): return -np.inf
        return lml.item()

    def fit(self, X_train, y_train, method='L-BFGS-B', disp=False, maxiter=100, model_name_for_print="SimpleGPR"):
        self.X_train = X_train
        self.y_train = y_train.reshape(-1,1)

        initial_params = self._get_params_array()
        if disp:
            print(f"  Starting {model_name_for_print} optimization...")
            print(f"    Initial params (ls, sf, noise): {np.round(initial_params,3)}")

        bounds = [(1e-5, 1e3), (1e-5, 1e3), (1e-5, 1e2)] # ls, sf, noise_std

        objective = lambda params_opt: -self.log_marginal_likelihood(params_opt, self.X_train, self.y_train)
        result = minimize(objective, initial_params, method=method, bounds=bounds, options={'disp': disp, 'maxiter': maxiter, 'ftol': 1e-7, 'gtol': 1e-5})

        if disp:
            print(f"  {model_name_for_print} optimization finished.")
            print(f"    Success: {result.success}, Message: {result.message}")
            print(f"    Optimized params (ls, sf, noise): {np.round(result.x,3)}")

        self._set_params_from_array(result.x)
        self.optimized_params_ = dict(zip(['ls', 'sf', 'noise'], np.round(result.x,4)))


        K_final = self.kernel(self.X_train, self.X_train)
        K_noisy_final = K_final + (self._noise_std**2) * np.eye(self.X_train.shape[0]) + 1e-7 * np.eye(self.X_train.shape[0])
        try:
            self.L_ = np.linalg.cholesky(K_noisy_final)
            self.alpha_ = scipy.linalg.solve_triangular(self.L_.T, scipy.linalg.solve_triangular(self.L_, self.y_train, lower=True, check_finite=False), lower=False, check_finite=False)
            self.fitted = True
        except np.linalg.LinAlgError:
            if disp:
                print("    Warning: Cholesky failed in SimpleGPR fit post-optimization. Using PINV for prediction.")
            K_noisy_inv = np.linalg.pinv(K_noisy_final)
            self.alpha_ = K_noisy_inv @ self.y_train
            self.L_ = None # Indicate Cholesky failed
            self.fitted = True
        return result

    def predict(self, X_star):
        if not self.fitted: raise RuntimeError("SimpleGPR must be fitted before prediction.")
        jitter = 1e-9

        K_star = self.kernel(X_star, self.X_train) # Covariance between test and train
        mean_star = K_star @ self.alpha_

        K_star_star_diag = np.diag(self.kernel(X_star, X_star)) # Covariance between test points themselves (diagonal)

        if self.L_ is not None:
            # v = L^-1 @ K_star.T
            v = scipy.linalg.solve_triangular(self.L_, K_star.T, lower=True, check_finite=False)
            var_reduction_diag = np.sum(v**2, axis=0) # diag(K_star @ K_noisy_inv @ K_star.T)
        else:
            # Fallback if Cholesky failed during fit
            K_noisy = self.kernel(self.X_train, self.X_train) + (self._noise_std**2) * np.eye(self.X_train.shape[0]) + 1e-7 * np.eye(self.X_train.shape[0])
            K_noisy_inv = np.linalg.pinv(K_noisy)
            var_reduction_diag = np.diag(K_star @ K_noisy_inv @ K_star.T)

        variance_star_diag = K_star_star_diag + (self._noise_std**2) - var_reduction_diag
        variance_star_diag = np.clip(variance_star_diag, jitter, np.inf) # Ensure positive

        return mean_star, variance_star_diag.reshape(-1,1)


def run_wine_experiment(verbose_optimization, seed):
    print("\n--- Starting WINE Dataset Experiment ---")
    try:
        wine_white_df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')
        wine_red_df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
    except Exception as e:
        print(f"Failed to load wine datasets: {e}. Skipping WINE experiment.")
        return

    X_S_raw = wine_white_df.drop('quality', axis=1).values
    y_S_raw = wine_white_df['quality'].values
    X_T_raw = wine_red_df.drop('quality', axis=1).values
    y_T_raw = wine_red_df['quality'].values

    scaler_S = StandardScaler()
    X_S = scaler_S.fit_transform(X_S_raw)

    scaler_T_all_wine = StandardScaler() # Specific scaler for wine target
    X_T_scaled_all = scaler_T_all_wine.fit_transform(X_T_raw)

    # Use 5% of Target data for training as per paper's setup for experiments section
    X_T_train, X_T_test, y_T_train, y_T_test = train_test_split(
        X_T_scaled_all, y_T_raw, train_size=0.05, random_state=seed
    )

    print(f"Source (White Wine): X_S {X_S.shape}, y_S {y_S_raw.shape}")
    print(f"Target (Red Wine) All: X_T_scaled_all {X_T_scaled_all.shape}, y_T {y_T_raw.shape}")
    print(f"Target Train: X_T_train {X_T_train.shape}, y_T_train {y_T_train.shape} (approx {X_T_train.shape[0]/X_T_raw.shape[0]*100:.1f}%)")
    print(f"Target Test: X_T_test {X_T_test.shape}, y_T_test {y_T_test.shape}")

    results = {}
    default_ls, default_sf, default_noise = 1.0, 1.0, 0.5
    default_b, default_mu = 1.0, 1.0 # Initial b and mu for AT-GP

    print("\n--- Training AT-GP Model (Wine) ---")
    atgp_model = ATGP(X_S, y_S_raw, X_T_train, y_T_train,
                      initial_length_scale=default_ls, initial_sigma_f=default_sf,
                      initial_b=default_b, initial_mu=default_mu,
                      initial_noise_S_std=default_noise, initial_noise_T_std=default_noise)
    atgp_model.fit(disp=verbose_optimization, maxiter=100)
    mean_atgp, _ = atgp_model.predict(X_T_test)
    results['AT-GP'] = nmse(y_T_test, mean_atgp)
    print(f"AT-GP NMSE (Wine): {results['AT-GP']:.4f}, Params: {atgp_model.optimized_params_}")

    print("\n--- Training No Transfer GP Model (Wine) ---")
    gp_no_transfer = SimpleGPR(initial_length_scale=default_ls, initial_sigma_f=default_sf, initial_noise_std=default_noise)
    gp_no_transfer.fit(X_T_train, y_T_train, disp=verbose_optimization, maxiter=100, model_name_for_print="No Transfer GP (Wine)")
    mean_no_transfer, _ = gp_no_transfer.predict(X_T_test)
    results['No Transfer GP'] = nmse(y_T_test, mean_no_transfer)
    print(f"No Transfer GP NMSE (Wine): {results['No Transfer GP']:.4f}, Params: {gp_no_transfer.optimized_params_}")

    print("\n--- Training Transfer All GP Model (Wine) ---")
    # For Transfer All, source data needs to be on the same scale as target data
    X_S_rescaled_for_All = scaler_T_all_wine.transform(scaler_S.inverse_transform(X_S)) # Inverse S-scale, then T-scale
    X_all_train = np.vstack((X_S_rescaled_for_All, X_T_train))
    y_all_train = np.concatenate((y_S_raw, y_T_train))

    gp_transfer_all = SimpleGPR(initial_length_scale=default_ls, initial_sigma_f=default_sf, initial_noise_std=default_noise)
    gp_transfer_all.fit(X_all_train, y_all_train, disp=verbose_optimization, maxiter=100, model_name_for_print="Transfer All GP (Wine)")
    mean_transfer_all, _ = gp_transfer_all.predict(X_T_test)
    results['Transfer All GP'] = nmse(y_T_test, mean_transfer_all)
    print(f"Transfer All GP NMSE (Wine): {results['Transfer All GP']:.4f}, Params: {gp_transfer_all.optimized_params_}")

    print("\n--- Summary of NMSE Results on WINE Dataset ---")
    for model_name, score in results.items():
        print(f"{model_name}: {score:.4f}")
    return results


def run_sarcos_experiment(verbose_optimization, seed):
    print("\n--- Starting SARCOS Dataset Experiment ---")
    try:
        # Use forward slashes or raw strings for paths
        input_file_path = 'C:/Disk D/Semester 4 2/Transfer Learning/Assignment/Assignment 3/Own Implementation/sarcos_inv.mat'
        output_file_path = 'C:/Disk D/Semester 4 2/Transfer Learning/Assignment/Assignment 3/Own Implementation/sarcos_inv_test.mat'

        sarcos_inputs_full = loadmat(input_file_path)['sarcos_inv']  # e.g., (44484, 21)
        sarcos_outputs_full = loadmat(output_file_path)['sarcos_inv_test']  # e.g., (4449, 7) - KEY CHANGED HERE as per your update

        print(f"Loaded sarcos_inputs_full: {sarcos_inputs_full.shape}")
        print(f"Loaded sarcos_outputs_full: {sarcos_outputs_full.shape}")

        # Determine the number of samples to work with (the minimum of the two)
        # This assumes the outputs in sarcos_inv_test.mat correspond to the *first* N
        # inputs in sarcos_inv.mat, where N is the number of rows in sarcos_inv_test.mat.
        num_common_samples = min(sarcos_inputs_full.shape[0], sarcos_outputs_full.shape[0])

        if num_common_samples == 0:
            print("No common samples found or one of the files is empty. Skipping SARCOS.")
            return
        
        print(f"Working with {num_common_samples} common samples.")

        # Subsample the larger array to match the smaller one
        sarcos_inputs_matched = sarcos_inputs_full[:num_common_samples, :]
        sarcos_outputs_matched = sarcos_outputs_full[:num_common_samples, :]

        if sarcos_outputs_matched.shape[1] < 2: # Ensure at least 2 joints in output
            print(f"SARCOS output data has only {sarcos_outputs_matched.shape[1]} joint(s). Need at least 2. Skipping.")
            return


    except FileNotFoundError:
        print("SARCOS .mat files not found. Please download 'sarcos_inv.mat' and 'sarcos_inv_test.mat' (or check paths). Skipping SARCOS experiment.")
        return
    except KeyError as e:
        print(f"KeyError loading SARCOS data: {e}. Make sure the keys ('sarcos_inv', 'sarcos_inv_test') are correct for your .mat files. Skipping SARCOS experiment.")
        return
    except Exception as e:
        print(f"Error loading SARCOS data: {e}. Skipping SARCOS experiment.")
        return

    # --- Subsampling for Source and Target tasks from the matched data ---
    # Paper: "one of the task as the target task and another as the source task"
    # Subsample for demonstration speed. Paper uses all source data.
    
    # We will select n_source_samples and n_target_total_samples from num_common_samples
    n_source_samples_desired = 1000 # Reduced for speed
    n_target_total_samples_desired = 1000 # Reduced for speed

    if n_source_samples_desired + n_target_total_samples_desired > num_common_samples:
        print(f"Desired samples ({n_source_samples_desired + n_target_total_samples_desired}) exceed available common samples ({num_common_samples}). Adjusting.")
        scale_factor = num_common_samples / (n_source_samples_desired + n_target_total_samples_desired)
        n_source_samples = int(n_source_samples_desired * scale_factor)
        n_target_total_samples = num_common_samples - n_source_samples
        # Ensure a minimum for target to allow splitting
        min_target_for_split = 20 # Need at least 2 for train (5% of 20 is 1, need 2)
        if n_target_total_samples < min_target_for_split and num_common_samples >= n_source_samples + min_target_for_split :
            n_target_total_samples = min_target_for_split
            n_source_samples = num_common_samples - n_target_total_samples
        elif n_target_total_samples < min_target_for_split: # Not enough even if source is minimal
            print(f"Not enough common samples ({num_common_samples}) to satisfy minimum target samples for split. Skipping.")
            return

        if n_source_samples <= 0 : # Ensure some source samples
            if num_common_samples > n_target_total_samples:
                n_source_samples = num_common_samples - n_target_total_samples
            else:
                print("Cannot allocate source samples. Skipping SARCOS.")
                return
        print(f"Adjusted: n_source_samples={n_source_samples}, n_target_total_samples={n_target_total_samples}")
    else:
        n_source_samples = n_source_samples_desired
        n_target_total_samples = n_target_total_samples_desired
    
    if n_source_samples <= 0 or n_target_total_samples <= 0:
        print("No samples allocated for source or target after adjustment. Skipping SARCOS.")
        return

    # Permutation on the common number of samples
    indices = np.random.RandomState(seed).permutation(num_common_samples)
    
    source_indices_from_perm = indices[:n_source_samples]
    target_indices_from_perm = indices[n_source_samples : n_source_samples + n_target_total_samples]

    # Select data using these valid indices from the matched arrays
    X_S_raw = sarcos_inputs_matched[source_indices_from_perm, :]
    y_S_raw = sarcos_outputs_matched[source_indices_from_perm, 0] # Joint 1 for source task

    X_T_raw_all = sarcos_inputs_matched[target_indices_from_perm, :]
    y_T_raw_all = sarcos_outputs_matched[target_indices_from_perm, 1] # Joint 2 for target task

    # --- Scaling and Splitting Target Data ---
    scaler_S = StandardScaler()
    X_S = scaler_S.fit_transform(X_S_raw)

    scaler_T_all_sarcos = StandardScaler()
    X_T_scaled_all = scaler_T_all_sarcos.fit_transform(X_T_raw_all)

    # Use 5% of Target data for training, ensure at least 2 samples for training
    # And enough samples for test
    if y_T_raw_all.shape[0] < 2: # y_T_raw_all now has n_target_total_samples
        print(f"Not enough samples in y_T_raw_all ({y_T_raw_all.shape[0]}) to split. Skipping SARCOS.")
        return

    train_size_target_actual = 0.05
    num_train_target = int(y_T_raw_all.shape[0] * train_size_target_actual)

    if num_train_target < 2:
        if y_T_raw_all.shape[0] >= 2: # If we have at least 2 total, use 2 for training
            num_train_target = 2
            train_size_target_actual = num_train_target / y_T_raw_all.shape[0]
        else: # Should have been caught by y_T_raw_all.shape[0] < 2
            print(f"Cannot get 2 training samples from target data of size {y_T_raw_all.shape[0]}. Skipping SARCOS.")
            return
    
    if y_T_raw_all.shape[0] - num_train_target < 1: # Ensure at least 1 test sample
        print(f"Not enough samples for testing after allocating {num_train_target} for training from {y_T_raw_all.shape[0]}. Skipping.")
        return


    X_T_train, X_T_test, y_T_train, y_T_test = train_test_split(
        X_T_scaled_all, y_T_raw_all, train_size=train_size_target_actual, random_state=seed
    )
    
    if X_T_train.shape[0] < 2: # Should be redundant due to above checks
        print("Not enough target training samples for SARCOS after split. Skipping.")
        return

    print(f"Source (SARCOS Joint 1): X_S {X_S.shape}, y_S {y_S_raw.shape}")
    print(f"Target Pool (SARCOS Joint 2): X_T_raw_all {X_T_raw_all.shape}, y_T_raw_all {y_T_raw_all.shape}")
    print(f"Target Train: X_T_train {X_T_train.shape}, y_T_train {y_T_train.shape} (train_size={train_size_target_actual*100:.1f}%)")
    print(f"Target Test: X_T_test {X_T_test.shape}, y_T_test {y_T_test.shape}")

    results = {}
    default_ls, default_sf, default_noise = 1.0, 1.0, 0.1 
    default_b, default_mu = 1.0, 1.0
    max_iter_sarcos = 30 # Reduced for speed

    print(f"\n--- Training AT-GP Model (SARCOS, max_iter={max_iter_sarcos}) ---")
    atgp_model = ATGP(X_S, y_S_raw, X_T_train, y_T_train,
                      initial_length_scale=default_ls, initial_sigma_f=default_sf,
                      initial_b=default_b, initial_mu=default_mu,
                      initial_noise_S_std=default_noise, initial_noise_T_std=default_noise)
    atgp_model.fit(disp=verbose_optimization, maxiter=max_iter_sarcos)
    mean_atgp, _ = atgp_model.predict(X_T_test)
    results['AT-GP'] = nmse(y_T_test, mean_atgp)
    print(f"AT-GP NMSE (SARCOS): {results['AT-GP']:.4f}, Params: {atgp_model.optimized_params_}")

    print(f"\n--- Training No Transfer GP Model (SARCOS, max_iter={max_iter_sarcos}) ---")
    gp_no_transfer = SimpleGPR(initial_length_scale=default_ls, initial_sigma_f=default_sf, initial_noise_std=default_noise)
    gp_no_transfer.fit(X_T_train, y_T_train, disp=verbose_optimization, maxiter=max_iter_sarcos, model_name_for_print="No Transfer GP (SARCOS)")
    mean_no_transfer, _ = gp_no_transfer.predict(X_T_test)
    results['No Transfer GP'] = nmse(y_T_test, mean_no_transfer)
    print(f"No Transfer GP NMSE (SARCOS): {results['No Transfer GP']:.4f}, Params: {gp_no_transfer.optimized_params_}")

    print(f"\n--- Training Transfer All GP Model (SARCOS, max_iter={max_iter_sarcos}) ---")
    X_S_rescaled_for_All = scaler_T_all_sarcos.transform(scaler_S.inverse_transform(X_S))
    X_all_train = np.vstack((X_S_rescaled_for_All, X_T_train))
    y_all_train = np.concatenate((y_S_raw, y_T_train))

    gp_transfer_all = SimpleGPR(initial_length_scale=default_ls, initial_sigma_f=default_sf, initial_noise_std=default_noise)
    gp_transfer_all.fit(X_all_train, y_all_train, disp=verbose_optimization, maxiter=max_iter_sarcos, model_name_for_print="Transfer All GP (SARCOS)")
    mean_transfer_all, _ = gp_transfer_all.predict(X_T_test)
    results['Transfer All GP'] = nmse(y_T_test, mean_transfer_all)
    print(f"Transfer All GP NMSE (SARCOS): {results['Transfer All GP']:.4f}, Params: {gp_transfer_all.optimized_params_}")

    print("\n--- Summary of NMSE Results on SARCOS Dataset ---")
    for model_name, score in results.items():
        print(f"{model_name}: {score:.4f}")
    return results

# The rest of your script (wine, sim_wifi, main block) remains the same.
# Make sure to replace the run_sarcos_experiment in your main script with this one.

def run_simulated_wifi_experiment(verbose_optimization, seed):
    print("\n--- Starting SIMULATED WiFi Localization Experiment ---")
    rng = np.random.RandomState(seed)
    n_features = 10
    n_source_samples = 500
    n_target_samples_all = 500 # Total target samples before splitting

    # Source task
    w_S = rng.randn(n_features, 1)
    X_S_raw = rng.rand(n_source_samples, n_features) * 10
    y_S_raw = X_S_raw @ w_S + rng.randn(n_source_samples, 1) * 0.5 # Source y

    # Target task (related but different)
    delta_w = rng.randn(n_features, 1) * 0.5 # Difference in weights
    w_T = w_S + delta_w
    # Target inputs might have a slight shift too
    X_T_raw_all = rng.rand(n_target_samples_all, n_features) * 10 + rng.rand(1, n_features) * 2
    y_T_raw_all = X_T_raw_all @ w_T + rng.randn(n_target_samples_all, 1) * 0.5 # Target y

    scaler_S = StandardScaler()
    X_S = scaler_S.fit_transform(X_S_raw)

    scaler_T_all_wifi = StandardScaler()
    X_T_scaled_all = scaler_T_all_wifi.fit_transform(X_T_raw_all)

    # 5% of target for training
    X_T_train, X_T_test, y_T_train, y_T_test = train_test_split(
        X_T_scaled_all, y_T_raw_all.ravel(), train_size=0.05, random_state=seed
    )
    if X_T_train.shape[0] < 2 :
        print("Not enough target training samples for WiFi Sim after split. Skipping.")
        return

    print(f"Source (Sim WiFi): X_S {X_S.shape}, y_S {y_S_raw.ravel().shape}")
    print(f"Target (Sim WiFi) All: X_T_scaled_all {X_T_scaled_all.shape}, y_T {y_T_raw_all.ravel().shape}")
    print(f"Target Train: X_T_train {X_T_train.shape}, y_T_train {y_T_train.shape} (approx {X_T_train.shape[0]/X_T_raw_all.shape[0]*100:.1f}%)")
    print(f"Target Test: X_T_test {X_T_test.shape}, y_T_test {y_T_test.shape}")

    results = {}
    default_ls, default_sf, default_noise = 1.0, 1.0, 0.2
    default_b, default_mu = 1.0, 1.0

    print("\n--- Training AT-GP Model (Sim WiFi) ---")
    atgp_model = ATGP(X_S, y_S_raw.ravel(), X_T_train, y_T_train,
                      initial_length_scale=default_ls, initial_sigma_f=default_sf,
                      initial_b=default_b, initial_mu=default_mu,
                      initial_noise_S_std=default_noise, initial_noise_T_std=default_noise)
    atgp_model.fit(disp=verbose_optimization, maxiter=100)
    mean_atgp, _ = atgp_model.predict(X_T_test)
    results['AT-GP'] = nmse(y_T_test, mean_atgp)
    print(f"AT-GP NMSE (Sim WiFi): {results['AT-GP']:.4f}, Params: {atgp_model.optimized_params_}")

    print("\n--- Training No Transfer GP Model (Sim WiFi) ---")
    gp_no_transfer = SimpleGPR(initial_length_scale=default_ls, initial_sigma_f=default_sf, initial_noise_std=default_noise)
    gp_no_transfer.fit(X_T_train, y_T_train, disp=verbose_optimization, maxiter=100, model_name_for_print="No Transfer GP (Sim WiFi)")
    mean_no_transfer, _ = gp_no_transfer.predict(X_T_test)
    results['No Transfer GP'] = nmse(y_T_test, mean_no_transfer)
    print(f"No Transfer GP NMSE (Sim WiFi): {results['No Transfer GP']:.4f}, Params: {gp_no_transfer.optimized_params_}")

    print("\n--- Training Transfer All GP Model (Sim WiFi) ---")
    X_S_rescaled_for_All = scaler_T_all_wifi.transform(scaler_S.inverse_transform(X_S))
    X_all_train = np.vstack((X_S_rescaled_for_All, X_T_train))
    y_all_train = np.concatenate((y_S_raw.ravel(), y_T_train))

    gp_transfer_all = SimpleGPR(initial_length_scale=default_ls, initial_sigma_f=default_sf, initial_noise_std=default_noise)
    gp_transfer_all.fit(X_all_train, y_all_train, disp=verbose_optimization, maxiter=100, model_name_for_print="Transfer All GP (Sim WiFi)")
    mean_transfer_all, _ = gp_transfer_all.predict(X_T_test)
    results['Transfer All GP'] = nmse(y_T_test, mean_transfer_all)
    print(f"Transfer All GP NMSE (Sim WiFi): {results['Transfer All GP']:.4f}, Params: {gp_transfer_all.optimized_params_}")

    print("\n--- Summary of NMSE Results on SIMULATED WiFi Dataset ---")
    for model_name, score in results.items():
        print(f"{model_name}: {score:.4f}")
    return results

In [18]:
if __name__ == '__main__':
    np.set_printoptions(precision=4, suppress=True)
    np.seterr(all='ignore') # Suppress common warnings during optimization
    verbose_opt = False  # Set to True for detailed optimization output from L-BFGS-B
    random_seed = 42 # For reproducibility

    sim_wifi_results = run_simulated_wifi_experiment(verbose_optimization=verbose_opt, seed=random_seed)

    print("\n\n--- OVERALL RESULTS SUMMARY ---")
    if sim_wifi_results:
        print("\nSimulated WiFi Dataset:")
        for model, nmse_val in sim_wifi_results.items(): print(f"  {model}: {nmse_val:.4f}")


--- Starting SIMULATED WiFi Localization Experiment ---
Source (Sim WiFi): X_S (500, 10), y_S (500,)
Target (Sim WiFi) All: X_T_scaled_all (500, 10), y_T (500,)
Target Train: X_T_train (25, 10), y_T_train (25,) (approx 5.0%)
Target Test: X_T_test (475, 10), y_T_test (475,)

--- Training AT-GP Model (Sim WiFi) ---
AT-GP NMSE (Sim WiFi): 0.0039, Params: {'ls': np.float64(192.1643), 'sf': np.float64(601.5818), 'b': np.float64(9.601), 'mu': np.float64(0.0), 'noise_S': np.float64(43.7325), 'noise_T': np.float64(0.3646), 'lambda': np.float64(0.9997)}

--- Training No Transfer GP Model (Sim WiFi) ---
No Transfer GP NMSE (Sim WiFi): 0.0040, Params: {'ls': np.float64(210.1262), 'sf': np.float64(366.5622), 'noise': np.float64(0.387)}

--- Training Transfer All GP Model (Sim WiFi) ---
Transfer All GP NMSE (Sim WiFi): 1.0114, Params: {'ls': np.float64(76.2268), 'sf': np.float64(162.6419), 'noise': np.float64(2.201)}

--- Summary of NMSE Results on SIMULATED WiFi Dataset ---
AT-GP: 0.0039
No Trans

In [19]:
if __name__ == '__main__':
    np.set_printoptions(precision=4, suppress=True)
    np.seterr(all='ignore') # Suppress common warnings during optimization
    verbose_opt = False  # Set to True for detailed optimization output from L-BFGS-B
    random_seed = 42 # For reproducibility

    # Run experiments for each dataset
    sarcos_results = run_sarcos_experiment(verbose_optimization=verbose_opt, seed=random_seed)
    print("\n\n--- OVERALL RESULTS SUMMARY ---")
    if sarcos_results:
        print("\nSARCOS Dataset (Subsampled, J1->J2):")
        for model, nmse_val in sarcos_results.items(): print(f"  {model}: {nmse_val:.4f}")


--- Starting SARCOS Dataset Experiment ---
Loaded sarcos_inputs_full: (44484, 28)
Loaded sarcos_outputs_full: (4449, 28)
Working with 4449 common samples.
Source (SARCOS Joint 1): X_S (1000, 28), y_S (1000,)
Target Pool (SARCOS Joint 2): X_T_raw_all (1000, 28), y_T_raw_all (1000,)
Target Train: X_T_train (50, 28), y_T_train (50,) (train_size=5.0%)
Target Test: X_T_test (950, 28), y_T_test (950,)

--- Training AT-GP Model (SARCOS, max_iter=30) ---
AT-GP NMSE (SARCOS): 1.0150, Params: {'ls': np.float64(118.1881), 'sf': np.float64(0.4545), 'b': np.float64(0.0), 'mu': np.float64(0.0), 'noise_S': np.float64(6.0924), 'noise_T': np.float64(0.1072), 'lambda': np.float64(1.0)}

--- Training No Transfer GP Model (SARCOS, max_iter=30) ---
No Transfer GP NMSE (SARCOS): 1.0206, Params: {'ls': np.float64(688.068), 'sf': np.float64(0.335), 'noise': np.float64(0.1084)}

--- Training Transfer All GP Model (SARCOS, max_iter=30) ---
Transfer All GP NMSE (SARCOS): 4.4362, Params: {'ls': np.float64(1.5076

In [20]:
if __name__ == '__main__':
    np.set_printoptions(precision=4, suppress=True)
    np.seterr(all='ignore') # Suppress common warnings during optimization
    verbose_opt = False  # Set to True for detailed optimization output from L-BFGS-B
    random_seed = 42 # For reproducibility

    sim_wine_results = run_wine_experiment(verbose_optimization=verbose_opt, seed=random_seed)

    print("\n\n--- OVERALL RESULTS SUMMARY ---")
    if sim_wine_results:
        print("\nSimulated Wine Dataset:")
        for model, nmse_val in sim_wine_results.items(): print(f"  {model}: {nmse_val:.4f}")


--- Starting WINE Dataset Experiment ---
Source (White Wine): X_S (4898, 11), y_S (4898,)
Target (Red Wine) All: X_T_scaled_all (1599, 11), y_T (1599,)
Target Train: X_T_train (79, 11), y_T_train (79,) (approx 4.9%)
Target Test: X_T_test (1520, 11), y_T_test (1520,)

--- Training AT-GP Model (Wine) ---
AT-GP NMSE (Wine): 0.7345, Params: {'ls': np.float64(102.3041), 'sf': np.float64(103.0598), 'b': np.float64(0.0003), 'mu': np.float64(3.0226), 'noise_S': np.float64(6.9137), 'noise_T': np.float64(0.5119), 'lambda': np.float64(0.9992)}

--- Training No Transfer GP Model (Wine) ---
No Transfer GP NMSE (Wine): 0.7379, Params: {'ls': np.float64(379.5052), 'sf': np.float64(58.9384), 'noise': np.float64(0.6133)}

--- Training Transfer All GP Model (Wine) ---
Transfer All GP NMSE (Wine): 1.0631, Params: {'ls': np.float64(38.023), 'sf': np.float64(37.9402), 'noise': np.float64(0.7059)}

--- Summary of NMSE Results on WINE Dataset ---
AT-GP: 0.7345
No Transfer GP: 0.7379
Transfer All GP: 1.0631
